# Download Images

In [3]:
from PIL import Image
import flickrapi
import requests

# Replace with your own API key and secret
api_key = '937c5fb63a6d88a2a148aec387c14272'
api_secret = 'b21c82310caf7c0f'

# Initialize the Flickr API client
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

# Use the 'flickr.photos.search' method to search for photos
# Here, we're searching for photos tagged with 'cat'
result = flickr.photos.search(tags='fashion')

# Extract the URLs of the photos
photo_urls = []
for photo in result['photos']['photo']:
    url = f"https://farm{photo['farm']}.staticflickr.com/{photo['server']}/{photo['id']}_{photo['secret']}.jpg"
    photo_urls.append(url)

# Print the list of URLs
print(photo_urls)

['https://farm66.staticflickr.com/65535/52754000013_038e7ceab5.jpg', 'https://farm66.staticflickr.com/65535/52753807230_e7425d1646.jpg', 'https://farm66.staticflickr.com/65535/52753620139_19f7ef09fd.jpg', 'https://farm66.staticflickr.com/65535/52752835642_2bfe44e656.jpg', 'https://farm66.staticflickr.com/65535/52753448389_b5981b7ed3.jpg', 'https://farm66.staticflickr.com/65535/52753597448_3774815deb.jpg', 'https://farm66.staticflickr.com/65535/52752542027_dc0430820e.jpg', 'https://farm66.staticflickr.com/65535/52753403785_3fb67fa67e.jpg', 'https://farm66.staticflickr.com/65535/52752442272_2251c44952.jpg', 'https://farm66.staticflickr.com/65535/52753464808_a2790440e9.jpg', 'https://farm66.staticflickr.com/65535/52753239640_c221fd73c0.jpg', 'https://farm66.staticflickr.com/65535/52752782376_6e92e1c26f.jpg', 'https://farm66.staticflickr.com/65535/52753099578_3b6036f206.jpg', 'https://farm66.staticflickr.com/65535/52752488601_1e6867d911.jpg', 'https://farm66.staticflickr.com/65535/52752703

In [4]:
images = []
urls = []

for url in photo_urls:
  try:
    images.append(Image.open(requests.get(url, stream=True).raw))
    urls.append(url)
  except:
    # print(f'{url} does not exist')
    pass

In [6]:
import torch
import clip
from PIL import Image
import requests
from io import BytesIO
import numpy as np
from transformers import CLIPProcessor, CLIPModel

# Get model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [7]:
import transformers

transformers.__version__

'4.18.0'

# Elasticsearch

In [1]:
try:
  import os
  import elasticsearch
  from elasticsearch import Elasticsearch
  import numpy as np
  import pandas as pd
  import sys
  import json
  from ast import literal_eval
  from tqdm import tqdm 
  import datetime
  from elasticsearch import helpers
  
except Exception as e:
  print(f"error: {e}")

In [2]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200}])
# Check if python is connected to elasticsearch
es.ping()

True

# CLIP Image Search

In [3]:
INDEX_NAME = 'clip-fashion'
VECTOR_DIMS = 512

In [4]:
# Retrieve the list of index names
index_names = es.indices.get_alias("*").keys()

# Print the list of index names
print(index_names)

dict_keys(['clip-fashion', 'patents-bert'])


In [14]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()
class Text(BaseModel):
    text: str

@app.post("/vectorize")
async def vectorize(text: Text):
    vector = model.encode_text(text.text)
    return {"vector": vector.tolist()}

In [19]:
%uvicorn main:app --reload

UsageError: Line magic function `%uvicorn` not found.


In [9]:
# Query a description
import pprint

text = clip.tokenize("a man").to(device)

with torch.no_grad():
    User_Query_Vector = model.encode_text(text)

    script_query = {"script_score": {"query": {"match_all": {}},
                                    "script": {"source": "cosineSimilarity(params.query_vector, doc['Image_vector']) ",
                                              "params": {"query_vector": np.array(model.encode_text(text).squeeze().tolist()) }
                                              }
                                  }
                  }

    response = es.search(index=INDEX_NAME,body={"size": 10, "query": script_query,
                                                    "_source": {"includes": ["Image_name"]}
                                                    }
                            )


    pprint.pprint(response)

RequestError: RequestError(400, 'search_phase_execution_exception', 'runtime error')

In [56]:
import requests

response = requests.get("http://ipecho.net/plain")
public_ip = response.text
print("Your public IP address is:", public_ip)
?

Your public IP address is: 77.129.211.10
